In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

def adicionar_conta_txt(conta: str):
    arquivo_txt = f'contas/contas_criadas.txt'
    os.makedirs(os.path.dirname(arquivo_txt), exist_ok=True)
    with open(arquivo_txt, 'a') as f:
        f.write(conta + '\n')
    print(f'Conta gravada em {arquivo_txt}')

def configurar_proxy(options, proxy: str):
    options.add_argument(f'--proxy-server={proxy}')
    return options

def validar_caminho_edgedriver(path: str):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"O arquivo '{path}' não foi encontrado. Verifique o caminho.")

def cadastrar_conta_papaya(driver, email):
    driver.get('https://www.papayaplay.com/account/signup.do')
    campo_usuario = driver.find_element(By.NAME, 'userid')
    campo_email = driver.find_element(By.ID, 'equalEmail')
    campo_confirma_email = driver.find_element(By.ID, 'confirmEmail')
    campo_senha = driver.find_element(By.NAME, 'pwd')
    campo_checkbox = driver.find_element(By.ID, 'custCheckbox')

    usuario_conta = email.split('@')[0]
    campo_usuario.send_keys(usuario_conta)
    campo_email.send_keys(email)
    campo_confirma_email.send_keys(email)
    campo_senha.send_keys(usuario_conta)
    if not campo_checkbox.is_selected():
        driver.execute_script("arguments[0].click();", campo_checkbox)

    esperar_resolucao_captcha(driver)

    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[2]/div/form/button'))
    )

    driver.execute_script("arguments[0].scrollIntoView(true);", button)
    button.click()

    return usuario_conta

def obter_email_temporario(driver, site_email_temporario: str):
    driver.get(site_email_temporario)
    email_element = driver.find_element(By.ID, 'email-input')
    email_temporario = email_element.get_attribute('value')
    print(f'Email gerado: {email_temporario}')
    return email_temporario

def clicar_email(driver):
    tbody_xpath = '/html/body/main/div[1]/div[3]/div[1]/table/tbody'
    tbody_element = driver.find_element(By.XPATH, tbody_xpath)
    tr_elements = tbody_element.find_elements(By.XPATH, './tr')

    for tr_element in tr_elements:
        try:
            element_id = tr_element.get_attribute('id')
            if element_id:
                print(f'ID encontrado: {element_id}')
                dynamic_tr_xpath = f'//*[@id="{element_id}"]'
                script = """
                    var element = arguments[0];
                    if (element) {
                        element.click();
                    }
                """
                driver.execute_script(script, tr_element)
                break
        except Exception as e:
            print(f'Erro ao encontrar o ID: {e}')

    WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.XPATH, f'//*[@id="{element_id}"]'))
    )

def clicar_link(driver):
    shadow_host_xpath = '//*[@id="body"]'
    shadow_host = driver.find_element(By.XPATH, shadow_host_xpath)

    script_shadow = """
        var shadowHost = arguments[0];
        var shadowRoot = shadowHost.shadowRoot;
        if (shadowRoot) {
            return Array.from(shadowRoot.querySelectorAll('a')).map(a => a.href);
        }
        return [];
    """
    links = driver.execute_script(script_shadow, shadow_host)

    if links:
        first_link = links[0]
        print(f"Primeiro link encontrado: {first_link}")
        driver.get(first_link)
    else:
        print("Nenhum link encontrado.")

def validar_email_apos_criar_conta_papaya(driver):
    clicar_email(driver)
    clicar_link(driver)

def validar_criacao_conta_papaya(driver):
    body_element = driver.find_element(By.ID, "portal")
    h2_elements = body_element.find_elements(By.XPATH, './h2')

    for h2_element in h2_elements:
        try:
            h2_value = h2_element.get_attribute('value')
            if h2_value == "Sign Up Completed":
                print("Conta cadastrada com sucesso")
                break
        except Exception as e:
            print(f'Erro ao cadastrar conta')

def esperar_resolucao_captcha(driver):
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe[title='reCAPTCHA']")))
    captcha = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))

    while True:
        time.sleep(2)
        class_name = "recaptcha-checkbox-checked"
        if class_name in captcha.get_attribute("class"):
            print("reCAPTCHA foi verificado com sucesso!")
            break
        else:
            print("reCAPTCHA ainda não foi verificado.")
    driver.switch_to.default_content()

def obter_profile(path, options):
    # Configura as opções do Edge para carregar o perfil
    options.add_argument(f'user-data-dir={path}')
    return options

def main():
    site_email_temporario = 'https://www.invertexto.com/gerador-email-temporario'
    edge_driver_path = r'msedgedriver.exe'  # Atualize o caminho para o msedgedriver
    validar_caminho_edgedriver(edge_driver_path)
    
    edge_options = EdgeOptions()
    
    # Adiciona o diretório do perfil padrão
    profile_path = r'C:\Users\Kaio\AppData\Local\Microsoft\Edge\User Data\Profile 1'  # Atualize o caminho do perfil
    edge_options = obter_profile(profile_path, edge_options)

    # Configure o Edge WebDriver com as opções corretas
    driver = webdriver.Edge(service=EdgeService(executable_path=edge_driver_path), options=edge_options)
    
    # Exemplo de uso do driver
    qtd_contas_criacao = 100
    contas_por_sessao = 4
    for i, _ in enumerate(range(qtd_contas_criacao), start=1):
        # Se a quantidade de contas criadas nesta sessão atingir o limite, reinicie o WebDriver
        if i % contas_por_sessao == 1 and i != 1:
            driver.quit()
            driver = webdriver.Edge(service=EdgeService(executable_path=edge_driver_path), options=edge_options)

        email_temporario = obter_email_temporario(driver, site_email_temporario)
        usuario_conta = cadastrar_conta_papaya(driver, email_temporario)

        driver.get(site_email_temporario + f"?email={email_temporario}")
        time.sleep(10)

        validar_email_apos_criar_conta_papaya(driver)
        validar_criacao_conta_papaya(driver)
        adicionar_conta_txt(usuario_conta)

        print("Quantidade de contas geradas", i)

    driver.quit()

if __name__ == '__main__':
    main()